In [8]:
import numpy as np
import time

In [2]:
def Discrete_RW_T(A):    # Calculate the degree vector (sum of non-zero elements in each row)
    degree_vector = np.array(A.sum(axis=1)).ravel()

    # Avoid division by zero by setting degree to 1 for nodes with no edges
    no_out_edges = degree_vector == 0
    degree_vector[no_out_edges] = 1.0

    # Create a diagonal matrix from the inverted degree vector
    degree_matrix = np.diag(1.0 / degree_vector)

    # Calculate the random walk Transition matrix: T = D^(-1) * A
    T = degree_matrix @ A
    np.fill_diagonal(T, no_out_edges, wrap=False)
    return T

In [3]:
def branching_tree_adjacency_matrix(c, d):
    # Total number of nodes in the tree
    total_nodes = sum(c ** i for i in range(d + 1))
    
    # Initialize an empty adjacency matrix
    adjacency_matrix = np.zeros((total_nodes, total_nodes), dtype=int)
    
    # Keep track of the current node index at each level
    current_index = 0
    
    # Loop through each level in the tree
    for level in range(d):
        # Number of nodes at the current level
        nodes_at_level = c ** level
        # Number of nodes at the next level
        nodes_at_next_level = c ** (level + 1)
        
        # For each node at the current level, connect it to `c` children at the next level
        for node in range(nodes_at_level):
            # Calculate the index of the first child at the next level
            parent_index = current_index + node
            first_child_index = current_index + nodes_at_level + node * c
            # Connect to each child (if within bounds)
            for child_offset in range(c):
                child_index = first_child_index + child_offset
                if child_index < current_index + nodes_at_level + nodes_at_next_level:
                    adjacency_matrix[parent_index, child_index] = 1
                    adjacency_matrix[child_index, parent_index] = 1

        # Move the current index to the start of the next level
        current_index += nodes_at_level
    
    return adjacency_matrix

In [4]:
def representative_indices(c, d):
    indices = []
    current_index = 0  # Start at the root node (index 0)
    
    for level in range(d + 1):
        indices.append(current_index)  # Add the first node of this level
        current_index += c ** level  # Move to the start of the next level
        
    return indices

In [5]:
def compute_probabilities(c, d, max_tau):
    # Initialize a 2D array to store probabilities for each level and time step
    probabilities = np.zeros((max_tau + 1, d + 1))
    
    # Set the initial condition: W(0, 0) = 1
    probabilities[0][0] = 1

    # Fill the probabilities table for each time step tau
    for tau in range(1, max_tau + 1):
        for l in range(d + 1):
            # Contribution from level l + 1
            if l + 1 <= d - 1:
                # Apply 1/(c+1) factor for levels other than d-1
                probabilities[tau][l] += probabilities[tau - 1][l + 1] * c / (c + 1)
            elif l + 1 == d:
                # At level d-1, keep the contribution as walks[tau - 1][l + 1] * c
                probabilities[tau][l] += probabilities[tau - 1][l + 1] * c
            
            # Contribution from level l - 1
            if l - 1 > 0:
                # Apply 1/(c+1) factor for levels other than 1
                probabilities[tau][l] += probabilities[tau - 1][l - 1] / (c + 1)
            elif l - 1 == 0:
                # At level 1, use 1/c factor for the root contribution
                probabilities[tau][l] += probabilities[tau - 1][l - 1] / c

    return probabilities

## Test Exactness

In [6]:

def test_transition_probabilities(c, d, max_tau):
    # Step 1: Create the adjacency matrix using branching tree parameters
    A = branching_tree_adjacency_matrix(c, d)

    # Step 2: Compute the random walk transition matrix T from A
    T = Discrete_RW_T(A)

    # Step 3: Initialize a list to store probabilities for each power (first method)
    first_method_probs = []

    # Step 4: Compute powers of T up to max_tau and get the first row
    level_indeces = representative_indices(c,d)
    for n in range(0, max_tau + 1):
        T_n = np.linalg.matrix_power(T, n)
        first_method_probs.append(T_n[0,level_indeces])

    # Step 5: Compute probabilities using the second method
    second_method_probs = compute_probabilities(c, d, max_tau)

    # Step 6: Compare both methods for each power and assert they are close
    for n in range(max_tau):
        if not np.allclose(first_method_probs[n], second_method_probs[n]):
            print(f"Discrepancy found at step {n + 1}")
            print("First method:", first_method_probs[n])
            print("Second method:", second_method_probs[n])
            return False

    print("All transition probabilities match for both methods!")
    return True



In [7]:
# Example usage:
c = 4  # e.g., number of children per node in the tree
d = 5  # depth of the tree
max_tau = 15  # maximum power to consider
test_transition_probabilities(c, d, max_tau)

All transition probabilities match for both methods!


True

## Test Speed

In [11]:
def speed_test_transition_probabilities(c, d, max_tau):
    # Step 1: Create the adjacency matrix using branching tree parameters
    A = branching_tree_adjacency_matrix(c, d)
    
    # Step 2: Compute the random walk transition matrix T from A
    T = Discrete_RW_T(A)

    # Timing the first method (matrix power method)
    start_time_first = time.time()
    first_method_probs = []
    level_indeces = representative_indices(c,d)
    for n in range(0, max_tau + 1):
        T_n = np.linalg.matrix_power(T, n)
        first_method_probs.append(T_n[0, level_indeces])
    end_time_first = time.time()
    time_first_method = end_time_first - start_time_first

    # Timing the second method (compute_probabilities)
    start_time_second = time.time()
    second_method_probs = compute_probabilities(c, d, max_tau)
    end_time_second = time.time()
    time_second_method = end_time_second - start_time_second

    # Print out the timing results
    print(f"Time taken by first method (matrix powers): {time_first_method:.6f} seconds")
    print(f"Time taken by second method (compute_probabilities): {time_second_method:.6f} seconds")

    # Comparing results for correctness as well (optional, can remove if only timing is needed)
    for n in range(max_tau):
        if not np.allclose(first_method_probs[n], second_method_probs[n]):
            print(f"Discrepancy found at step {n + 1}")
            print("First method:", first_method_probs[n])
            print("Second method:", second_method_probs[n])
            return False

    # Print comparison summary
    if time_first_method < time_second_method:
        print("Matrix power method is faster.")
    elif time_second_method < time_first_method:
        print("Compute probabilities method is faster.")
    else:
        print("Both methods take the same time.")

    return time_first_method, time_second_method


In [16]:
# Example usage:
c = 3  # number of children per node in the tree
d = 5  # depth of the tree
max_tau = 15  # maximum power to consider
speed_test_transition_probabilities(c, d, max_tau)

Time taken by first method (matrix powers): 0.032501 seconds
Time taken by second method (compute_probabilities): 0.000325 seconds
Compute probabilities method is faster.


(0.0325009822845459, 0.0003254413604736328)